## Summary

In this notebook:

- I created a DataFrame based on **"La Banque Sonore des Dialectes Bretons"**, containing the columns:
  - `br`: the original Breton sentence,
  - `fr`: the French translation,
  - `city`: the municipality associated with the speaker.

- There were some issues due to poor scraping of the Banque Sonore website: in some cases, the `br` column contained a dialectal form, while the `fr` column contained the standardized version of Breton.  
  ➤ This issue has been fixed.

- I added the following columns to the DataFrame:
  - `br_processed`: the cleaned version of the Breton text,
  - `transcription`: the transcription of the audio using the **Vosk** model,
  - `wer`: the Word Error Rate,
  - `cer`: the Character Error Rate.

- I made an evaluation of the Whisper Small model on 2,000 audio samples which is about 27% of the whole dataset.

- I computed:
  - the average WER and CER by **municipality**,
  - the average WER and CER by **department**,
  - and the overall WER and CER for the entire dataset.

In [16]:
import os
import wave
import json
import jiwer
import audio

import sys

import whisper

In [17]:
import pandas as pd

In [18]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("/home/ouassim/Downloads/0000.parquet",columns=["br","fr","city"])
# df = pd.read_parquet("/home/wazemour/Stage-Breton/Data/Banque sonore des Dialectes Bretonnes/0000.parquet",columns=["br","fr","city"])

In [19]:
def fix_row(row):
    if 'peurunvan :' in row['fr']:
        # Extract Breton text after "peurunvan :"
        new_text = row['fr'].split('peurunvan :', 1)[1].strip()
        # Replace both Br and Fr columns with this new_text
        row['br'] = new_text
        row['fr'] = new_text
    return row

df = df.apply(fix_row, axis=1)

In [20]:
df

,br,fr,city
0,Pell zo n'eo ket o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
1,N'oc'h ket bet pell zo o welet ac'hanon.,Vous n'êtes pas venus me voir depuis longtemps.,22107
2,Pell zo n'eo ket bet o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
3,Ma... ma zi ne vo ket gwerzhet.,Ma... ma maison ne sera pas vendue.,22107
4,Hon... hon ti ne vo ket gwerzhet.,Notre... notre maison ne sera pas vendue.,22107
...,...,...,...
7286,Ne zeue ket ag ar vro Kemper. Ne zeue ket ag a...,Il ne venait pas du pays de Quimper. Il ne ven...,56247
7287,Bout a oa linad razh d'en-dro ar feunteun. Bou...,Il y avait des orties tout autour de la fontai...,56247
7288,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247
7289,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247


In [21]:
# dataset_path = "/home/wazemour/Stage-Breton/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio"
dataset_path = "/home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio"

In [22]:
model = whisper.load_model("tiny")

In [23]:

from load_ground_truth import load_ground_truth_dict

from transcriber import transcribe_audio

from evaluate_wer_cer import evaluate_wer_cer

In [24]:
def generate_transcription(row):
    file_id = row['file']
    utt_name = f"utt_{file_id:05d}.wav"
    full_path = os.path.join(dataset_path, utt_name)
    return transcribe_audio(full_path, model)

In [27]:
small_df = df.sample(n=2000, random_state=42)
small_df

,br,fr,city
1057,ur c'harzh,une haie,29032
2654,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168
4401,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230
5843,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293
7177,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175
...,...,...,...
981,skarzhañ,vider,29032
1732,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,Mets tes mains dans ta poche ! Retire-les d'en...,29150
6185,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...","Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",56076
2354,"Oh nann, mes me a gred lar mar vije bet chomet...","Oh nann, mes me a gred lar mar vije bet chomet...",29153


In [28]:
# Step 1: Filter where city starts with '22'
# df_22 = df[df['city'].astype(str).str.startswith('22')].copy()

# Step 2: Add a 'file' column with the original index
small_df['file'] = small_df.index

small_df['transcription'] = small_df.apply(generate_transcription, axis=1)

# Optional: Reset index for a clean new DataFrame
small_df = small_df.reset_index(drop=True)

Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_01057.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:02.040]  f quo
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_02654.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:02.880]  Mori, as you do, benenne internus
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_04401.wav...


...

[00:00.000 --> 00:03.700]  ja, da shouting, da jegie protestar, da jegöraproma, Daha reiai tak semei spo whomon desu城
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_02354.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:02.600]  un hanno me da la chemacheUSAS
[00:02.600 --> 00:05.080]  shad
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_05555.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:05.280]  L �est sandwich
[00:05.500 --> 00:09.820]  когда丟距離詢 szix


In [29]:
small_df

,br,fr,city,file,transcription
0,ur c'harzh,une haie,29032,1057,f quo
1,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168,2654,"Mori, as you do, benenne internus"
2,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230,4401,Ronaldo bele War War War Story M Metal Red Ph...
3,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293,5843,tilwilibu teru win
4,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175,7177,ť nuha mid- altat
...,...,...,...,...,...
1995,skarzhañ,vider,29032,981,Scazen!
1996,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,Mets tes mains dans ta poche ! Retire-les d'en...,29150,1732,"Lactazora, no batabosh? Ten nemi istu stahaul"
1997,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...","Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",56076,6185,"ja, da shouting, da jegie protestar, da jegör..."
1998,"Oh nann, mes me a gred lar mar vije bet chomet...","Oh nann, mes me a gred lar mar vije bet chomet...",29153,2354,un hanno me da la chemacheUSAS shad


In [30]:
from filter_char import filter_out_chars
from normalizer import normalize_sentence
from utils import pre_process


def process_br_text(br):
    PUNCTUATION = '<>.?!,;:«»“”"()[]/…–—•'
    br = filter_out_chars(br, PUNCTUATION + '*')
    br = normalize_sentence(br, autocorrect=True)
    br = pre_process(br).replace('-', ' ').lower()
    return br

In [31]:
small_df['br_processed'] = small_df['br'].apply(process_br_text)

In [32]:
from jiwer import wer, cer

# Compute WER and CER for each row
small_df['wer'] = small_df.apply(lambda row: wer(row['br_processed'], row['transcription']), axis=1)
small_df['cer'] = small_df.apply(lambda row: cer(row['br_processed'], row['transcription']), axis=1)


In [33]:
small_df = small_df[['br', 'br_processed', 'fr', 'transcription', 'city', 'wer', 'cer', 'file']]
small_df

,br,br_processed,fr,transcription,city,wer,cer,file
0,ur c'harzh,ur c'harzh,une haie,f quo,29032,1.00,0.900000,1057
1,Mari zo deuet a-benn enno antronoz.,mari zo deuet a benn enno antronoz,Marie est venue vers eux le lendemain.,"Mori, as you do, benenne internus",29168,1.00,0.558824,2654
2,"An amzer gwechall, ne oa... ne oa ket, eu... n...",an amzer gwechall ne oa ne oa ket eu ne oa ket...,"Autrefois, c'était... c'était, euh... ça n'éta...",Ronaldo bele War War War Story M Metal Red Ph...,29230,1.00,0.892857,4401
3,Piv zo bet o troc'hiñ ar wezenn ?,piv zo bet o troc'hiñ ar wezenn,Qui a coupé l'arbre ?,tilwilibu teru win,29293,1.00,0.774194,5843
4,"Pa oufemp ar wirionez, n'he lavarfemp ket.",pa oufemp ar wirionez n'he lavarfemp ket,"Si nous savions la vérité, nous ne la dirions ...",ť nuha mid- altat,56175,1.00,0.775000,7177
...,...,...,...,...,...,...,...,...
1995,skarzhañ,skarzhañ,vider,Scazen!,29032,1.00,0.750000,981
1996,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,lak da zornoù e barzh da boch tenn anezhe er m...,Mets tes mains dans ta poche ! Retire-les d'en...,"Lactazora, no batabosh? Ten nemi istu stahaul",29150,1.00,0.636364,1732
1997,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",nend nend ay ket ganin bremañ c'hwi me am eus ...,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...","ja, da shouting, da jegie protestar, da jegör...",56076,1.00,0.902439,6185
1998,"Oh nann, mes me a gred lar mar vije bet chomet...",oh nann mes me a gred lâr mar vije bet chomet ...,"Oh nann, mes me a gred lar mar vije bet chomet...",un hanno me da la chemacheUSAS shad,29153,0.95,0.772727,2354


In [35]:
# Assuming your DataFrame has columns: 'city', 'wer', and 'cer'
city_avg = small_df.groupby('city')[['wer', 'cer']].mean()

# Optional: round to 2 decimal places
city_avg = city_avg.round(2)

# Print each city's average WER and CER
for city_code, row in city_avg.iterrows():
    print(f"Average WER and CER for {city_code}:\nWER: {row['wer']}\nCER: {row['cer']}\n")

Average WER and CER for 22107:
WER: 1.05
CER: 0.98

Average WER and CER for 22167:
WER: 1.09
CER: 0.88

Average WER and CER for 22181:
WER: 1.0
CER: 0.84

Average WER and CER for 22230:
WER: 1.01
CER: 0.79

Average WER and CER for 22331:
WER: 1.07
CER: 0.77

Average WER and CER for 22336:
WER: 0.99
CER: 0.83

Average WER and CER for 22386:
WER: 1.06
CER: 0.8

Average WER and CER for 29005:
WER: 1.0
CER: 0.78

Average WER and CER for 29031:
WER: 1.69
CER: 1.25

Average WER and CER for 29032:
WER: 1.0
CER: 0.8

Average WER and CER for 29058:
WER: 0.99
CER: 0.71

Average WER and CER for 29080:
WER: 1.0
CER: 0.76

Average WER and CER for 29082:
WER: 1.0
CER: 0.75

Average WER and CER for 29091:
WER: 1.0
CER: 0.91

Average WER and CER for 29105:
WER: 0.99
CER: 0.74

Average WER and CER for 29122:
WER: 1.01
CER: 0.82

Average WER and CER for 29136:
WER: 1.0
CER: 0.88

Average WER and CER for 29139:
WER: 1.05
CER: 0.85

Average WER and CER for 29146:
WER: 1.0
CER: 0.81

Average WER and CER fo

In [36]:
# Filter for cities starting with '22'
small_df_22 = small_df[small_df['city'].str.startswith('22')]
avg_22 = small_df_22[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '22'
small_df_29 = small_df[small_df['city'].str.startswith('29')]
avg_29 = small_df_29[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '56'
small_df_56 = small_df[small_df['city'].str.startswith('56')]
avg_56 = small_df_56[['wer', 'cer']].mean().round(4)



print("Evaluation for --Côtes-d'Armor-- dialects 22xxx :")
print(f"WER: {avg_22['wer']}")
print(f"CER: {avg_22['cer']}\n")

print("Evaluation for --Finistère-- dialects 29xxx :")
print(f"WER: {avg_29['wer']}")
print(f"CER: {avg_29['cer']}\n")

print("Evaluation for --Morbihan-- dialects 56xxx :")
print(f"WER: {avg_56['wer']}")
print(f"CER: {avg_56['cer']}")

Evaluation for --Côtes-d'Armor-- dialects 22xxx :
WER: 1.0381
CER: 0.8457

Evaluation for --Finistère-- dialects 29xxx :
WER: 1.0668
CER: 0.8548

Evaluation for --Morbihan-- dialects 56xxx :
WER: 1.0365
CER: 0.8505


In [37]:
# Assuming 'wer' and 'cer' columns exist
average_wer = small_df['wer'].mean()
average_cer = small_df['cer'].mean()

# Optional: round them
average_wer = round(average_wer, 2)
average_cer = round(average_cer, 2)

print(f"Overall Average WER: {average_wer}")
print(f"Overall Average CER: {average_cer}")

Overall Average WER: 1.06
Overall Average CER: 0.85
